<span style="font-size: 14pt">MIPT, Advanced ML, Autumn 2017</span>

<span style="font-size: 16pt"> HW #10: Recommender Systems

<span style="color:blue; font-size: 12pt">Andrey Saitgalin </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'"> andrey.saitgalin@gmail.com </span>

<h1 align="center">Organization Info</h1> 

* Дедлайн **18 декабря 23:59** для всех групп.
* В качестве решения задания нужно прислать ноутбук с подробными комментариями (<span style='color:red'> без присланного решения результат контеста не будет засчитан </span>).
* <span style='color:red'>Название команды в контесте должно соответствовать шаблону: НомерГруппы_Имя_Фамилия, например, 494_Ivan_Ivanov</span>.

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall <номер_группы> <фамилия>``, к примеру -- ``ML2017_Aut_fall 401 ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb``, к примеру -- ``ivanov_401_task10.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall Question <Содержание вопроса>``


--------
- **PS1**: Мы используем автоматические фильтры, и просто не найдем ваше дз, если вы не аккуратно его подпишите.
- **PS2**: Напоминаем, что дедлайны жесткие, письма пришедшие после автоматически удаляются =( чтобы соблазна не было.

<h1 align="center">Contest (100%)</h1> 
* Ссылка на контест: https://www.kaggle.com/t/f37413eb448145acba822f3a8c43946d

In [1]:
import pandas as pd
from os import path

DATA_DIR = '/home/egiby/Learning/Current/machine-learning/practice/data/contest100'
KAGGLE_DATA = 'data'
DATASET = 'ml-20m'

In [2]:
train_ids = pd.read_csv(path.join(DATA_DIR, KAGGLE_DATA, 'train.csv'))
train_ids.dtypes

rating     float64
movieId      int64
userId      uint64
dtype: object

In [3]:
train_ids.head()

,rating,movieId,userId
0,3.0,2158,10953099004185006703
1,3.5,12959,10953099004185006703
2,3.5,7808,10953099004185006703
3,4.0,9049,4762492897681260571
4,3.0,12363,2110017552365950661


In [4]:
test_ids = pd.read_csv(path.join(DATA_DIR, KAGGLE_DATA, 'test.csv'))
test_ids.dtypes

movieId     int64
userId     uint64
dtype: object

In [5]:
test_ids.head()

,movieId,userId
0,939,3128775406573976678
1,19,6058877902987009152
2,1213,2787275058272728238
3,1141,14199414671932452776
4,2935,5958052843391945803


In [6]:
print(test_ids.userId.min(), test_ids.userId.max())
print(train_ids.userId.min(), train_ids.userId.max())
# ура, они наконец одинаковые!

58385675418643 18446653389538581776
58385675418643 18446653389538581776


In [7]:
print("Unique user count", len(train_ids["userId"].unique()))
print("Unique movie count", len(train_ids["movieId"].unique()))

Unique user count 138493
Unique movie count 18345


## Хочется разделить на train и test и посмотреть, насколько все хорошо

In [5]:
import numpy as np


train_ids["random"] = np.random.random(size=(train_ids.shape[0], 1))
idx = train_ids.sort_values(by="random") \
    .reset_index() \
    .groupby("userId") \
    .head(n=7)["index"]

mask = train_ids.index.isin(idx)
train_df = train_ids[~mask]
test_df = train_ids[mask]

In [6]:
train_df[["userId", "movieId", "rating"]].to_csv("ml20m_train.csv", header=False, index=False)
test_df[["userId", "movieId", "rating"]].to_csv("ml20m_test.csv", header=False, index=False)

In [7]:
test_df.head()

,rating,movieId,userId,random
476,1.0,1714,13227923070531244354,0.000534
490,2.0,1270,8212772436416091663,0.006719
984,4.0,3792,13719044709052513250,0.006219
1033,2.0,379,1433873131833809595,0.001197
1108,3.5,286,6498419348948971207,0.002700


In [1]:
!./vw --version

8.4.0


In [12]:
!awk -F"," '{printf "%f |u %d |i %d\n", $3,$1,$2}' < ml20m_train.csv | \
  ./vw /dev/stdin -b 18 -q ui --rank 30 --l2 0.001 --learning_rate 0.015 \
    --passes 5 --decay_learning_rate 0.97 --power_t 0 \
    --loss_function squared -f movielens.reg -k --cache_file movielens.cache --threads

creating quadratic features for pairs: ui 
using l2 regularization = 0.001
final_regressor = movielens.reg
Num weight bits = 18
learning rate = 0.015
initial_t = 1
power_t = 0
decay_learning_rate = 0.97
creating cache_file = movielens.cache
Reading datafile = /dev/stdin
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
7.913301 7.913301            1            1.0   3.0000   0.1869       63
9.081711 10.250121            2            2.0   3.5000   0.2984       63
10.423429 11.765148            4            4.0   4.0000   0.3387       63
9.942033 9.460636            8            8.0   4.5000   0.3959       63
10.205175 10.468317           16           16.0   2.5000   0.7053       63
7.334792 4.464409           32           32.0   4.0000   0.9697       63
5.623727 3.912662           64           64.0   3.0000   1.5551       63
3.897234 2.170740          128          128.0   4.0

In [13]:
!awk -F"," '{printf "%f |u %d |i %d\n", $3,$1,$2}' < ml20m_test.csv | \
  ./vw /dev/stdin -i movielens.reg -t -p ml20m_test_predictions.csv

creating quadratic features for pairs: ui 
only testing
predictions = ml20m_test_predictions.csv
Num weight bits = 18
learning rate = 10
initial_t = 1
power_t = 0.5
using no cache
Reading datafile = /dev/stdin
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
6.794386 6.794386            1            1.0   1.0000   3.6066       63
4.610961 2.427535            2            2.0   2.0000   3.5581       63
2.792235 0.973510            4            4.0   2.0000   3.0494       63
1.487212 0.182189            8            8.0   4.0000   4.2255       63
0.940563 0.393914           16           16.0   4.0000   3.7545       63
0.943373 0.946183           32           32.0   4.0000   3.3181       63
0.630813 0.318253           64           64.0   3.5000   3.3431       63
0.802550 0.974286          128          128.0   2.5000   3.4713       63
0.794405 0.786260          256          256.

In [16]:
import numpy as np
import pandas as pd

In [17]:
!head ml20m_test_predictions.csv

3.606604
3.558055
3.080317
3.049382
3.063125
4.040788
3.803318
4.225533
2.523062
3.532927


In [18]:
ratings = pd.read_csv('ml20m_test_predictions.csv', header=None)

In [19]:
test_df = pd.read_csv('ml20m_test.csv', header=None)

In [20]:
true_ratings = test_df[2]
true_ratings.head()

0    1.0
1    2.0
2    4.0
3    2.0
4    3.5
Name: 2, dtype: float64

In [22]:
from sklearn.metrics import mean_squared_error

mean_squared_error(ratings, true_ratings)

0.97352047451054036

## Похоже, оно работает. Теперь обучение на всей выборке и отправка в контест

In [14]:
import numpy as np


test_ids['rating'] = np.zeros(test_ids.shape[0])

In [12]:
test_ids.shape

(969451, 3)

In [ ]:
train_ids[["userId", "movieId", "rating"]].to_csv("ml20m_train.csv", header=False, index=False)

In [15]:
test_ids[["userId", "movieId", "rating"]].to_csv("ml20m_test.csv", header=False, index=False)

In [16]:
!awk -F"," '{printf "%f |u %d |i %d\n", $3,$1,$2}' < ml20m_train.csv | \
  ./vw /dev/stdin -b 18 -q ui --rank 30 --l2 0.001 --learning_rate 0.015 \
    --passes 5 --decay_learning_rate 0.97 --power_t 0 \
    --loss_function squared -f movielens.reg -k --cache_file movielens.cache --threads

creating quadratic features for pairs: ui 
using l2 regularization = 0.001
final_regressor = movielens.reg
Num weight bits = 18
learning rate = 0.015
initial_t = 1
power_t = 0
decay_learning_rate = 0.97
creating cache_file = movielens.cache
Reading datafile = /dev/stdin
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
7.913301 7.913301            1            1.0   3.0000   0.1869       63
9.081711 10.250121            2            2.0   3.5000   0.2984       63
10.423429 11.765148            4            4.0   4.0000   0.3387       63
9.942033 9.460636            8            8.0   4.5000   0.3959       63
10.205175 10.468317           16           16.0   2.5000   0.7053       63
7.334792 4.464409           32           32.0   4.0000   0.9697       63
5.623727 3.912662           64           64.0   3.0000   1.5551       63
3.897234 2.170740          128          128.0   4.0

In [19]:
!awk -F"," '{printf "%f |u %d |i %d\n", $3,$1,$2}' < ml20m_test.csv | \
  ./vw /dev/stdin -i movielens.reg -t -p ml20m_test_predictions.csv

creating quadratic features for pairs: ui 
only testing
predictions = ml20m_test_predictions.csv
Num weight bits = 18
learning rate = 10
initial_t = 1
power_t = 0.5
using no cache
Reading datafile = /dev/stdin
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
16.195406 16.195406            1            1.0   0.0000   4.0244       63
10.687369 5.179333            2            2.0   0.0000   2.2758       63
12.083152 13.478934            4            4.0   0.0000   3.4476       63
12.932878 13.782604            8            8.0   0.0000   4.1141       63
12.972806 13.012733           16           16.0   0.0000   3.7787       63
12.201732 11.430659           32           32.0   0.0000   3.7546       63
12.486272 12.770812           64           64.0   0.0000   3.4999       63
12.761836 13.037400          128          128.0   0.0000   3.6119       63
12.579160 12.396484          

In [20]:
!head ml20m_test_predictions.csv

4.024352
2.275815
3.882221
3.447641
3.615921
3.455373
3.631771
4.114142
3.548950
4.246598


In [22]:
predictions = pd.read_csv('ml20m_test_predictions.csv', header=None)

In [24]:
answer = pd.read_csv(path.join(DATA_DIR, KAGGLE_DATA, 'baseline.csv'))

In [26]:
answer['rating'] = predictions

In [29]:
answer.to_csv('predictions.csv', index=False)

In [30]:
!head predictions.csv

Id,rating
1,4.0243519999999995
2,2.275815
3,3.882221
4,3.4476410000000004
5,3.6159209999999997
6,3.4553730000000002
7,3.631771
8,4.114142
9,3.54895
